In [1]:
# generate abbreviations using some fixed ordering combinations
# returns dictionary of abbrev:rubber
import re
def gen_abbrevs(s):
    
    w = s.split(' ')

    brand = w[0]
    series = w[1]
    combos = []
    if len(w) > 4:
        combos = []
    elif len(w) == 2:
        combos.append(series)
    elif len(w) >= 3:
        name = w[2]
        if len(w) == 3:
            combos.append(series+' '+name)
            if '-' in name:
                combos.append(name)
                combos.append(name.replace('-',''))
                combos.append(series + ' ' + name.replace('-',''))
            if name.isnumeric():
                if series.lower() != 'skyline':
                    combos.append(series[0]+name)
                combos.append(series+name)
            if series.lower()=='tenergy' \
                or (brand.lower()=='donic' \
                and re.search(r'\d', name) \
                and re.search(r'.*[a-zA-Z].*',name)):
                combos.append(name)
        elif len(w) == 4:
            quant = w[3]
            combos.append(name+'-'+quant)
            combos.append(name+quant)
            combos.append(name+' '+quant)    
            combos.append(series + ' ' + name + '-' + quant)
            combos.append(series + ' ' + name + ' ' + quant)
            combos.append(series + ' ' + name + '' + quant)
            if name.isnumeric():
                combos.append(series[0]+name+quant)
                combos.append(series[0]+name+'-'+quant)

    w_brand = []
    for c in combos:
        w_brand.append(brand + ' ' + c)
    combos = combos + w_brand

    s_compact = '-'.join(w)
    
    return {c:s_compact for c in combos}, combos, s_compact


In [8]:
# examples
print(gen_abbrevs('Donic Baracuda')[1])
print(gen_abbrevs('Donic Acuda S2')[1])
print(gen_abbrevs('Butterfly Tenergy 05')[1])

['Baracuda', 'Donic Baracuda']
['Acuda S2', 'S2', 'Donic Acuda S2', 'Donic S2']
['Tenergy 05', 'T05', 'Tenergy05', '05', 'Butterfly Tenergy 05', 'Butterfly T05', 'Butterfly Tenergy05', 'Butterfly 05']


In [3]:
# do this on the list of rubbers we are interested in for now

import csv

with open('rub_master_list.csv', newline='') as f:
    reader = csv.reader(f)
    rub_list = list(reader)

rub_dict = {}
rub_name_list = []

for rub in rub_list:
    d, _, n = gen_abbrevs(rub[0])
    rub_dict.update(d)
    rub_name_list.append(n)
    
# other miscellaneous items
with open('rub_list_misc.csv') as f:
    reader = csv.reader(f)
    rub_list_misc = list(reader)
rub_dict.update({k:v for (k,v) in rub_list_misc})

rub_name_list = rub_name_list + [v for (_,v) in rub_list_misc]
rub_name_list = set(rub_name_list)

# make all lower case
rub_dict = {k.lower(): v for k, v in rub_dict.items()}

In [4]:
# check to see if substitutions make sense
import re

regex = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in rub_dict.keys()) + r')(?!\w)',re.IGNORECASE)

def sub_rubber_names(s):
    if isinstance(s, str):
        return regex.sub(lambda m: rub_dict[m.group(0).lower()],s)
    else:
        return ''

sub_rubber_names('i love evolution elp and t05 and el-p and T05FX and tensor rubbers and ten and z1 and speed')

'i love Tibhar-Evolution-EL-P and Butterfly-Tenergy-05 and Tibhar-Evolution-EL-P and Butterfly-Tenergy-05-FX and Tensor-rubbers and Butterfly-Tenergy and Donic-Bluestorm-Z1 and speed'

In [5]:
# save to csv file
with open('rub_dict.csv', 'w') as f:
    for key in rub_dict.keys():
        f.write("%s,%s\n"%(key,rub_dict[key]))
        
with open('rub_name_list.csv', 'w') as f:
    for rub in rub_name_list:
        f.write("%s\n"%(rub))

In [6]:
len(rub_dict)

2834